**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

In [2]:
pwd

/minikube-host/training/notebooks


In [12]:
DOMAIN=$(hostname -i  | sed s/\\./-/g).nip.io
echo $DOMAIN

192-168-49-2.nip.io


In [87]:
git clone https://github.com/goharbor/harbor-helm/

Cloning into 'harbor-helm'...
remote: Enumerating objects: 10, done.        
remote: Counting objects: 100% (10/10), done.        
remote: Compressing objects: 100% (10/10), done.        
remote: Total 3660 (delta 0), reused 3 (delta 0), pack-reused 3650
Receiving objects: 100% (3660/3660), 15.06 MiB | 5.63 MiB/s, done.
Resolving deltas: 100% (2365/2365), done.


In [178]:
(cd harbor-helm; git checkout values.yaml; git checkout v1.6.0)

Updated 1 path from the index
Previous HEAD position was 1eb2687 Merge pull request #828 from ywk253100/210111_1.5.3
HEAD is now at 7d3d33f Merge pull request #863 from ywk253100/210225_1.6.0


In [179]:
ls harbor-helm

CONTRIBUTING.md  LICENSE    cert  docs       test
Chart.yaml       README.md  conf  templates  values.yaml


In [180]:
sed -i s/harbor\\.domain/$DOMAIN/ harbor-helm/values.yaml
cat harbor-helm/values.yaml | grep --color=always -E ".*$DOMAIN.*|$"

expose:
  # Set the way how to expose the service. Set the type as "ingress",
  # "clusterIP", "nodePort" or "loadBalancer" and fill the information
  # in the corresponding section
  type: ingress
  tls:
    # Enable the tls or not.
    # Delete the "ssl-redirect" annotations in "expose.ingress.annotations" when TLS is disabled and "expose.type" is "ingress"
    # Note: if the "expose.type" is "ingress" and the tls
    # is disabled, the port must be included in the command when pull/push
    # images. Refer to https://github.com/goharbor/harbor/issues/5291
    # for the detail.
    enabled: true
    # The source of the tls certificate. Set it as "auto", "secret"
    # or "none" and fill the information in the corresponding section
    # 1) auto: generate the tls certificate automatically
    # 2) secret: read the tls certificate from the specified secret.
    # The tls certificate can be generated manually or by cert manager
    # 3) none: configure no tls certificate for the ingress

In [181]:
grep  $DOMAIN harbor-helm/values.yaml 

      core: core.192-168-49-2.nip.io
      notary: notary.192-168-49-2.nip.io
externalURL: https://core.192-168-49-2.nip.io


In [182]:
helm install my-harbor harbor-helm

NAME: my-harbor
LAST DEPLOYED: Tue Mar  2 17:40:27 2021
NAMESPACE: default
STATUS: deployed
REVISION: 1
TEST SUITE: None
NOTES:
Please wait for several minutes for Harbor deployment to complete.
Then you should be able to visit the Harbor portal at https://core.192-168-49-2.nip.io
For more details, please visit https://github.com/goharbor/harbor


In [187]:
kubectl get all

NAME                                                  READY   STATUS    RESTARTS   AGE
pod/my-harbor-harbor-chartmuseum-7df65db446-lpk7x     1/1     Running   0          30s
pod/my-harbor-harbor-core-84ffd7948d-58t64            0/1     Running   0          30s
pod/my-harbor-harbor-database-0                       1/1     Running   0          30s
pod/my-harbor-harbor-jobservice-556db5457b-r68rk      0/1     Running   0          30s
pod/my-harbor-harbor-notary-server-65c64cdd97-ck6v9   1/1     Running   0          30s
pod/my-harbor-harbor-notary-signer-948b9d8d-x9qhx     1/1     Running   0          30s
pod/my-harbor-harbor-portal-54899cd5d4-bl8d9          1/1     Running   0          30s
pod/my-harbor-harbor-redis-0                          1/1     Running   0          30s
pod/my-harbor-harbor-registry-6f46fcf4b7-8cx8g        2/2     Running   0          30s
pod/my-harbor-harbor-trivy-0                          1/1     Running   0          30s

NAME                                     T

In [184]:
kubectl get pv

NAME                                       CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                               STORAGECLASS   REASON   AGE
pvc-3ddc5f66-3f4d-4dc0-9c6f-f6e4d2881533   1Gi        RWO            Delete           Bound    default/my-harbor-harbor-jobservice                 standard                12s
pvc-3e77fb26-e37f-4ca7-91a8-41e1ebc4e361   5Gi        RWO            Delete           Bound    default/my-harbor-harbor-chartmuseum                standard                12s
pvc-7ab0a586-fc69-4b33-be0c-7251724b6124   1Gi        RWO            Delete           Bound    default/data-my-harbor-harbor-redis-0               standard                11s
pvc-bf1c42ca-73bd-46bc-a065-517a68dbd1ec   5Gi        RWO            Delete           Bound    default/my-harbor-harbor-registry                   standard                12s
pvc-d0c54b7e-65b8-428b-aae8-b71eca6bebf2   1Gi        RWO            Delete           Bound    default/database-data-my-harbo

## Ingress

The ingress controller must be running.

If not already showing an ingress, use `minikube addons enable ingress` outside this

In [53]:
kubectl get svc -n kube-system | grep ingress

ingress-nginx-controller-admission   ClusterIP   10.104.203.7   <none>        443/TCP                  2m47s


In [55]:
docker images | grep goharbor

goharbor/chartmuseum-photon                             v2.2.0     7d2b0430a45d   7 days ago      165MB
goharbor/redis-photon                                   v2.2.0     dab1804560b2   7 days ago      68.9MB
goharbor/trivy-adapter-photon                           v2.2.0     658abcdfe7e3   7 days ago      120MB
goharbor/notary-server-photon                           v2.2.0     5117cee34e76   7 days ago      101MB
goharbor/notary-signer-photon                           v2.2.0     5d8879c476a8   7 days ago      98.4MB
goharbor/harbor-registryctl                             v2.2.0     6833a758fcfb   7 days ago      128MB
goharbor/registry-photon                                v2.2.0     c5797c8f62f1   7 days ago      77.2MB
goharbor/harbor-jobservice                              v2.2.0     62b4ae79b159   7 days ago      163MB
goharbor/harbor-core                                    v2.2.0     049ef19b7080   7 days ago      148MB
goharbor/harbor-portal                                  v2.2.

## Let's try it

In [4]:
git clone https://github.com/endocode/minimal-go-container-from-scratch.git

Cloning into 'minimal-go-container-from-scratch'...
remote: Enumerating objects: 20, done.        
remote: Total 20 (delta 0), reused 0 (delta 0), pack-reused 20        
Unpacking objects: 100% (20/20), 1.53 MiB | 4.99 MiB/s, done.


In [13]:
docker build -t $DOMAIN/minimal-go-container-from-scratch minimal-go-container-from-scratch/micro

Sending build context to Docker daemon  4.625MB
Step 1/4 : FROM scratch
 ---> 
Step 2/4 : USER 12345
 ---> Using cache
 ---> f963d8c102a2
Step 3/4 : ADD micro /micro
 ---> 6ef3e7b12395
Step 4/4 : CMD ["/micro"]
 ---> Running in 4092ad85c1fe
Removing intermediate container 4092ad85c1fe
 ---> ccd1f145b7f1
Successfully built ccd1f145b7f1
Successfully tagged 192-168-49-2.nip.io/minimal-go-container-from-scratch:latest


In [16]:
docker images

REPOSITORY                                              TAG        IMAGE ID       CREATED          SIZE
192-168-49-2.nip.io/minimal-go-container-from-scratch   latest     ccd1f145b7f1   30 seconds ago   4.62MB
goharbor/chartmuseum-photon                             v2.2.0     7d2b0430a45d   9 days ago       165MB
goharbor/redis-photon                                   v2.2.0     dab1804560b2   9 days ago       68.9MB
goharbor/trivy-adapter-photon                           v2.2.0     658abcdfe7e3   9 days ago       120MB
goharbor/notary-server-photon                           v2.2.0     5117cee34e76   9 days ago       101MB
goharbor/notary-signer-photon                           v2.2.0     5d8879c476a8   9 days ago       98.4MB
goharbor/harbor-registryctl                             v2.2.0     6833a758fcfb   9 days ago       128MB
goharbor/registry-photon                                v2.2.0     c5797c8f62f1   9 days ago       77.2MB
goharbor/harbor-jobservice                          

In [17]:
docker push 192-168-49-2.nip.io/minimal-go-container-from-scratch

Using default tag: latest
The push refers to repository [192-168-49-2.nip.io/minimal-go-container-from-scratch]
Get https://192-168-49-2.nip.io/v2/: x509: certificate is valid for ingress.local, not 192-168-49-2.nip.io


: 1

In [1]:
helm ls

NAME     	NAMESPACE	REVISION	UPDATED                               	STATUS  	CHART       	APP VERSION
my-harbor	default  	1       	2021-03-02 17:40:27.64361189 +0000 UTC	deployed	harbor-1.6.0	2.2.0      


In [2]:
kubectl get pv,pvc

NAME                                                        CAPACITY   ACCESS MODES   RECLAIM POLICY   STATUS   CLAIM                                               STORAGECLASS   REASON   AGE
persistentvolume/pvc-3ddc5f66-3f4d-4dc0-9c6f-f6e4d2881533   1Gi        RWO            Delete           Bound    default/my-harbor-harbor-jobservice                 standard                45h
persistentvolume/pvc-3e77fb26-e37f-4ca7-91a8-41e1ebc4e361   5Gi        RWO            Delete           Bound    default/my-harbor-harbor-chartmuseum                standard                45h
persistentvolume/pvc-7ab0a586-fc69-4b33-be0c-7251724b6124   1Gi        RWO            Delete           Bound    default/data-my-harbor-harbor-redis-0               standard                45h
persistentvolume/pvc-bf1c42ca-73bd-46bc-a065-517a68dbd1ec   5Gi        RWO            Delete           Bound    default/my-harbor-harbor-registry                   standard                45h
persistentvolume/pvc-d0c54b7e-65b8-428b-

In [175]:
kubectl delete pvc $(kubectl get pvc -o jsonpath='{.items[*].metadata.name}')

persistentvolumeclaim "data-my-harbor-harbor-redis-0" deleted
persistentvolumeclaim "data-my-harbor-harbor-trivy-0" deleted
persistentvolumeclaim "database-data-my-harbor-harbor-database-0" deleted
persistentvolumeclaim "my-harbor-harbor-chartmuseum" deleted
persistentvolumeclaim "my-harbor-harbor-jobservice" deleted
persistentvolumeclaim "my-harbor-harbor-registry" deleted


In [ ]:
kubectl delete pv $(kubectl get pv -o jsonpath='{.items[*].metadata.name}')

In [ ]:
kubectl get pvc,pv